In [8]:
%matplotlib inline
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

import pulp

from typing import Optional, List,Tuple,Dict

# Phone Ranking Problem using UTA and AHP

Łukasz Andryszewski 151930

Choosing a new phone is often quite a daunting task, with many different models from different companies and varying prices.

In [9]:
data = pd.read_csv("./data/dataset.csv")
prefs = pd.read_csv("./data/preferences.csv")

In [10]:
data

,name,price-cost,ram-gain,screen-gain,disk-gain,oled-gain,os-gain,foldable-gain
0,Huawei P30,1699,6.000,6.10,128,1,3,0
1,Huawei P20 Lite,899,4.000,5.80,64,0,3,0
2,Iphone 15,4699,6.000,6.70,128,1,0,0
3,Iphone 12,2499,4.000,6.10,64,1,0,0
4,Motorola Razr Ultra,3999,8.000,6.90,256,1,3,1
5,Motorola Razr 40,2999,8.000,6.90,256,1,3,1
6,Samsung Galaxy Z Flip 5,5599,8.000,6.70,256,1,3,1
7,Motorola Razr 8,2657,8.000,6.67,256,1,3,1
8,Nokia 2660 Flip,299,4.000,2.80,128,1,1,1
9,Huawei P40,1949,8.000,6.10,128,1,2,0


In [4]:
crit_types = [True if crit.split("-")[1]=="gain" else False for crit in data.drop(columns="name").columns ]
crit_types

[False, True, True, True, True, True, True]

In [5]:
data.columns = list(map(lambda s: s.split("-")[0],data.columns))
data

,name,price,ram,screen,disk,oled,os,foldable
0,Huawei P30,1699,6.000,6.10,128,1,3,0
1,Huawei P20 Lite,899,4.000,5.80,64,0,3,0
2,Iphone 15,4699,6.000,6.70,128,1,0,0
3,Iphone 12,2499,4.000,6.10,64,1,0,0
4,Motorola Razr Ultra,3999,8.000,6.90,256,1,3,1
5,Motorola Razr 40,2999,8.000,6.90,256,1,3,1
6,Samsung Galaxy Z Flip 5,5599,8.000,6.70,256,1,3,1
7,Motorola Razr 8,2657,8.000,6.67,256,1,3,1
8,Nokia 2660 Flip,299,4.000,2.80,128,1,1,1
9,Huawei P40,1949,8.000,6.10,128,1,2,0


In [6]:
prefs

,id,preferred_to
0,0,1
1,4,6
2,8,11
3,3,2
4,3,9


In [11]:
def show_preferences(prefs,names,id_column="id",pref_column = "preferred_to"):
    for i,j in zip(prefs[id_column],prefs[pref_column]):
        print(f"{names[i]} is preferred to {names[j]}")

In [12]:
show_preferences(prefs,data["name"])

Huawei P30 is preferred to Huawei P20 Lite
Motorola Razr Ultra is preferred to Samsung Galaxy Z Flip 5
Nokia 2660 Flip is preferred to Nokia 3310
Iphone 12 is preferred to Iphone 15
Iphone 12 is preferred to Huawei P40
